In [23]:
#r "nuget: Microsoft.SemanticKernel, 1.33.0"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);


Installed Packages Microsoft.SemanticKernel, 1.33.0

In [24]:
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.33.0-preview"

Installed Packages Microsoft.SemanticKernel.Connectors.InMemory, 1.33.0-preview

In [25]:
using Microsoft.Extensions.VectorData;
public sealed class KnowledgeBaseSnippet
{
    [VectorStoreRecordKey]
    public required string Key { get; set; }

    [VectorStoreRecordData]
    public string? Text { get; set; }

    [VectorStoreRecordData]
    public string? ReferenceDescription { get; set; }

    [VectorStoreRecordData]
    public string? ReferenceLink { get; set; }

    [VectorStoreRecordVector(1536)]
    public ReadOnlyMemory<float> TextEmbedding { get; set; }
}

In [26]:
string collectionName = "MyKnowledgebase";
string rootPath = "knowledgebase";

In [34]:
using Microsoft.SemanticKernel.Connectors.InMemory;
using Microsoft.SemanticKernel.Data;
#pragma warning disable SKEXP0020
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0010

// Define vector store
var vectorStore = new InMemoryVectorStore();

builder
    .AddAzureOpenAITextEmbeddingGeneration(
        "text-embedding-ada-002",
        embeddingEndpoint,
        embeddingApiKey
    )
    .AddInMemoryVectorStore()
    .AddInMemoryVectorStoreRecordCollection<string, KnowledgeBaseSnippet>(collectionName)
    .AddVectorStoreTextSearch<KnowledgeBaseSnippet>(
        new TextSearchStringMapper((result) => (result as KnowledgeBaseSnippet)!.Text!),
        new TextSearchResultMapper((result) =>
        {
            // Create a mapping from the Vector Store data type to the data type returned by the Text Search.
            // This text search will ultimately be used in a plugin and this TextSearchResult will be returned to the prompt template
            // when the plugin is invoked from the prompt template.
            var castResult = result as KnowledgeBaseSnippet;
            return new TextSearchResult(value: castResult!.Text!) { Name = castResult.ReferenceDescription, Link = castResult.ReferenceLink };
        }));

In [35]:
var kernel = builder.Build();

In [37]:
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Embeddings;
#pragma warning disable SKEXP0001

var vectorStoreRecordCollection = kernel.GetRequiredService<IVectorStoreRecordCollection<string, KnowledgeBaseSnippet>>();
var textEmbeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();
var vectorStoreTextSearch = kernel.GetRequiredService<VectorStoreTextSearch<KnowledgeBaseSnippet>>();

In [38]:
var embedding = await textEmbeddingGenerationService.GenerateEmbeddingAsync("Hello, World!");
embedding

[ -0.0026431093, -0.0061312397, -0.0068117715, -0.032820355, -0.00452828, 0.00393483, -0.021415796, -0.008521165, -0.0017368082, -0.022718806, 0.03279455, 0.009024307, -0.017455164, -0.015597409, 0.013713851, 0.009217824, 0.02693746, -0.009398439, 0.0059764264, 0.0095661525 ... (1516 more) ]

In [39]:
await vectorStoreRecordCollection.CreateCollectionIfNotExistsAsync(System.Threading.CancellationToken.None);


In [41]:
var markdownFiles = Directory.GetFiles(rootPath, "*.md", SearchOption.AllDirectories);
foreach (var markdownFile in markdownFiles)
{
    var text = File.ReadAllText(markdownFile);
    var embedding = await textEmbeddingGenerationService.GenerateEmbeddingAsync(text);
    var kbRecord = new KnowledgeBaseSnippet
    {
        Key = Guid.NewGuid().ToString(),
        Text = text,
        ReferenceDescription = $"{new FileInfo(markdownFile).Name}",
        ReferenceLink = $"{new Uri(new FileInfo(markdownFile).FullName).AbsoluteUri}",
        TextEmbedding = embedding
    };
    var key = vectorStoreRecordCollection.UpsertAsync(kbRecord);
    Console.WriteLine($"Upserted record for {markdownFile} with key: {key}");

}

Upserted record for knowledgebase\ingress-certificate-expired-warning-troubleshooting.md with key: System.Threading.Tasks.Task`1[System.String]
Upserted record for knowledgebase\ingress-certificate-expired-warning.md with key: System.Threading.Tasks.Task`1[System.String]
Upserted record for knowledgebase\k8s-pod-crashloopbackoff.md with key: System.Threading.Tasks.Task`1[System.String]


In [44]:
#pragma warning disable SKEXP0001
var results = await vectorStoreTextSearch.GetTextSearchResultsAsync("how to solve Warning of Ingress Certificate that was Expired",searchOptions: new TextSearchOptions() { Top = 1 });
await foreach (var result in results.Results)
{
    Console.WriteLine($"Result: {result.Name} - {result.Link}");
    Console.WriteLine(result.Value);
    Console.WriteLine("-----------------------------------------------------------------");
}


Result: ingress-certificate-expired-warning.md - file:///c:/Users/tamirdresher/source/repos/td-semantic-kernel-intro/notebooks/knowledgebase/ingress-certificate-expired-warning.md
# Resolving `IngressCertificateExpiredWarning` in Kubernetes

## Issue
The `IngressCertificateExpiredWarning` alert indicates that a TLS certificate used by an Ingress resource is expired or nearing expiration. This can cause HTTPS traffic to fail.

## Causes
- Let’s Encrypt or another certificate issuer has not renewed the certificate.
- The Kubernetes Certificate Manager (`cert-manager`) is not functioning correctly.
- The certificate secret is misconfigured or missing.

## Solution

### 1. Check Certificate Status
Run:
```sh
kubectl get certificate -A
```
Look for expired certificates and note the associated `SecretName`.

### 2. Check Cert-Manager Logs (if using cert-manager)
```sh
kubectl logs -n cert-manager deploy/cert-manager
```
Look for renewal failures.

### 3. Manually Renew the Certificate (if ne